<a id="top"></a>
# Brain Tumor Segmentation Demo using the Intel® Distribution of OpenVINO™ toolkit 

## Check for latest version
<br><div class=danger><b>Important: Before proceeding, please run the following cell to ensure that you are running the most recent version of this sample.</b></div>

In [ ]:
# Show the current status of this and all documents with ability to update
from qarpo.catalog import DemoCatalog
import os
status = DemoCatalog(os.getcwd(), "Demo").ShowRepositoryControls()

## Introduction

This sample application demonstrates how a smart video IoT solution may be created using Intel® hardware and software tools to perform brain tumor segmentation.  This solution segments brain tumors from raw MRI scans.  The result is a colorized overlay of the original MRI scan highlighting brain tumor matter.

### Key concepts
This sample application includes an example for the following:
- Application:
  - Video and image input is supported using OpenCV
  - OpenCV is used to draw bounding boxes around detected objects, labels, and other information
  - Visualization of the resulting bounding boxes in the output
- Intel® DevCloud for the Edge:
  - Submitting inference as jobs that are performed on different edge compute nodes (rather than on the development node hosting this Jupyter* notebook)
  - Monitoring job status
  - Viewing results and assessing performance for hardware on different compute nodes
- [Intel® Distribution of OpenVINO™ toolkit](https://software.intel.com/openvino-toolkit):
  - Create the necessary Intermediate Representation (IR) files for the inference model using the [Model Optimizer](http://docs.openvinotoolkit.org/latest/_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html)
  - Run an inference application on multiple hardware devices using the [Inference Engine](http://docs.openvinotoolkit.org/latest/_docs_IE_DG_Deep_Learning_Inference_Engine_DevGuide.html)


### Application background

To perform brain tumor segmentation (BraTS) the U-Net architecture is used to segment brain tumors from raw MRI scans. With relatively little data a U-Net model can be trained to accurately predict where tumors exist. 

The Dice coefficient (the standard metric for the BraTS dataset used in the study) for our model is about 0.82-0.88.  [Menze et al. reported](http://ieeexplore.ieee.org/document/6975210/) that expert neuroradiologists manually segmented these tumors with a cross-rater Dice score of 0.75-0.85, meaning that the model’s predictions are on par with what expert physicians have made. The below MRI brain scans show highlighted brain tumor matter segmented using a trained model. 

![](images/figure1.png)

### What is U-Net?
Since its introduction two years ago, the [U-Net](https://arxiv.org/pdf/1505.04597) architecture has been used to create deep learning models for segmenting [nerves](https://github.com/jocicmarko/ultrasound-nerve-segmentation) in ultrasound images, [lungs](https://www.kaggle.com/c/data-science-bowl-2017#tutorial) in CT scans, and even [interference](https://github.com/jakeret/tf_unet) in radio telescopes.

The architecture of U-Net is designed like an [auto-encoder](https://en.wikipedia.org/wiki/Autoencoder). It has an encoding path (“contracting”) paired with a decoding path (“expanding”) which gives it the `U` shape.  However, in contrast to the autoencoder, U-Net predicts a pixel-wise segmentation map of the input image rather than classifying the input image as a whole. For each pixel in the original image, it asks the question: “To which class does this pixel belong?” This flexibility allows U-Net to predict different parts of the tumor simultaneously.

![](images/unet.png)

## Brain tumor segmentation application
The brain tumor segmentation application uses the Intel® Distribution of OpenVINO™ toolkit to perform inference on a raw MRI input image to segment brain tumor matter within the image.  We will setup, run, and view the results for this application for several different hardware devices (CPU. GPU, etc.) available on the compute nodes within the Intel® DevCloud for the Edge.  To accomplish this, we will be performing the following tasks:

1. Use the [Model Optimizer](http://docs.openvinotoolkit.org/latest/_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html) to create the inference model IR files needed to perform inference
2. Create the job file used to submit running inference on compute nodes
3. Submit jobs for different compute nodes and monitor the job status until complete
4. View results and assess performance 

### How it works
At startup the brain tumor segmentation application configures itself by parsing the command line arguments.  Once configured, the application loads the specified inference model's IR files into the [Inference Engine](http://docs.openvinotoolkit.org/latest/_docs_IE_DG_Deep_Learning_Inference_Engine_DevGuide.html) and runs inference on the supplied raw MRI images to segment brain tumors.  The result is a combined image comprising a set of three images: The MRI input, ground truth, and the predicted output from inference with Dice score.  The combined image is also annotated with the amount of time to do inference.

To run the application on the Intel® DevCloud for the Edge, a job is submitted to an edge compute node with a hardware accelerator such as Intel® HD Graphics GPU, Intel® Movidius™ Neural Compute Stick 2 and and Intel® Arria® 10 FPGA.  After inference on the input is completed, the output is stored in the appropriate `results/<JobID>/` directory.  The results are then viewed within this Jupyter* Notebook using the `outputHTML` image display utility.

The application and inference code for this sample is already implemented in the two Python* files: [`healthcare_openvino.py`](./healthcare_openvino.py) and supporting [`argparser.py`](./argparser.py) and [`settings.py`](./settings.py).

The following sections will guide you through configuring and running the brain tumor segmentation application.

## Prerequisites

### Install Keras*
The Python* source code depends upon the ['keras`](https://keras.io/) module which is not present by default and must be installed before running this sample.

Run the following cell to test for and install `keras`.

<br><div class=note><i><b>Note: </b>Installation will take several minutes to complete</i></div>

In [ ]:
try:
    # Test whether keras package is installed
    import keras
    print("keras already installed")
except:
    # Exception while importing keras, try to install it using pip module
    print("Installing keras")
    !python3 -m pip install keras

### Install psutils
The Python* source code depends upon the ['psutils`](https://pypi.org/project/psutil/) module which is not present by default and must be installed before running this sample.

Run the following cell to test for and install `psutils`.

<br><div class=note><i><b>Note: </b>Installation will take several minutes to complete</i></div>

In [ ]:
try:
    # Test whether psutil package is installed
    import psutil
    print("psutil already installed")
except:
    # Exception while importing psutil, try to install it using pip module
    print("Installing psutil")
    !python3 -m pip install psutil

### Create the IR files for the inference model

The Intel® Distribution of OpenVINO™ toolkit includes the [Model Optimizer](http://docs.openvinotoolkit.org/latest/_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html) used to convert and optimize trained models into the Intermediate Representation (IR) model files, and the [Inference Engine](http://docs.openvinotoolkit.org/latest/_docs_IE_DG_Deep_Learning_Inference_Engine_DevGuide.html) that uses the IR model files to run inference on hardware devices.  The IR model files can be created from trained models from popular frameworks (e.g. Caffe\*, Tensorflow*, etc.).  For this sample, the model is supplied in the file `/data/Healthcare_app/data/saved_model_frozen.pb`.

The `saved_model_frozen.pb` file will need to be optimized using the Model Optimizer to create the necessary IR files.  We will be running the inference model on different hardware devices which have different requirements on the precision of the model (see [Inference Engine Supported Model Formats](https://docs.openvinotoolkit.org/latest/_docs_IE_DG_supported_plugins_Supported_Devices.html#supported_model_formats) for details).  For our purposes, we will focus on the use of the two most common precisions, FP32 and FP16.

For this model, we will run the Model Optimizer using the format:
```bash
/opt/intel/openvino/deployment_tools/model_optimizer/mo.py \
    --input_model <path_to_model> \
    --input_shape=[<N,H,W,C>] \
    --data_type <data_precision> \
    --output_dir <path_to_output_directory> \
    --model_name <name>
```

The input arguments are as follows:
- **--input_model** : The model's input *.pb* file
- **--input_shape** : The model's input data shape
 - **N** = Batch size
 - **H** = Height
 - **W** = Width
 - **C** = Number of channels
- **--data_type** : The model's data type and precision (e.g. FP16, FP32, INT8, etc.)
- **--output_dir** : Output directory where to store the generated IR model files
- **--model_name** : Output model name to use

The input shape, scaling values, and mean values we will be using are specific to the model topology being used.  Using the appropriate values for the model that we will use, The complete command will look like the following:
```bash
!/opt/intel/openvino/deployment_tools/model_optimizer/mo.py \
    --input_model /data/Healthcare_app/data/saved_model_frozen.pb \
    --input_shape=[1,144,144,4] \
    --data_type <data_precision> \
    --output_dir models/<data_precision> \
    --model_name saved_model
```
We will run the command twice, first with <*data_precision*> set to `FP16` and then `FP32` to get all the IR files we will need to run inference on different devices.

<br><div class=note><i><b>Note: </b>More information on how to use Model Optimizer to convert TensorFlow* models as well as specifying input shape and scaling parameters for common model topologies may be found at:[Converting a TensorFlow* Model](https://docs.openvinotoolkit.org/latest/_docs_MO_DG_prepare_model_convert_model_Convert_Model_From_TensorFlow.html)
</i></div>

Run the following cell to use the Model Optimizer to create the `FP16` and `FP32` model IR files:

In [ ]:
# Create FP16 IR files
!python3 /opt/intel/openvino/deployment_tools/model_optimizer/mo.py \
    --input_model /data/Healthcare_app/data/saved_model_frozen.pb \
    --input_shape=[1,144,144,4] \
    --data_type FP16  \
    --output_dir models/FP16  \
    --model_name saved_model

# Create FP32 IR files
!python3 /opt/intel/openvino/deployment_tools/model_optimizer/mo.py \
    --input_model /data/Healthcare_app/data/saved_model_frozen.pb \
    --input_shape=[1,144,144,4] \
    --data_type FP32  \
    --output_dir models/FP32  \
    --model_name saved_model

# find all resulting IR files
!echo "\nAll IR files that were created:"
!find ./models -name "*.xml" -o -name "*.bin"

<br><div class=tip><i><b>Tip: </b>The '!' at the beginning of a line is a special Jupyter* Notebook command that allows you to run shell commands as if you are at a command line. The above command will also work in a terminal (with the '!' removed).</i></div>

As shown above from the output of the last `!find...` command, the required sets of IR model files (`*.xml` and `*.bin`) have been created.  

### Run the demo

Running the next cell will display an interactive user interface allowing you to submit jobs to run the demo on multiple edge compute nodes selected by hardware devices, view the output of each job, and compare performance results across jobs.

To run a job:
1. Select the desired option in the **Target node** list
2. Select the desired device in the **Target architecture** list
3. Click the **Submit** button

After the **Submit** button is clicked, a tab will appear for the new job with a label in the format "*status*: *JobID*".  Once the job status appears as "Done", the **Display output** button may be clicked to view the output for the job.

After one or more jobs are done, the performance results for each job may be plotted by clicking the **Plot results** button.  Results for each job will be potted as bar graphs for **inference time** and **frames per second**.  Lower values mean better performance for **inference time** and higher values mean better performance for **frames per second**. When comparing results, please keep in mind that some architectures are optimized for highest performance, others for low power or other metrics.

Run the next cell to begin the interactive demo.

In [ ]:
# import necessary libraries
import json
import qarpo

# load job configurations for demo
with open('job_config.json') as json_file:  
    data = json.load(json_file)
    
# create and run the user job interface
job_interface = qarpo.Interface(data)
job_interface.displayUI()

## Next steps
- The complete [Brain Tumor Segmentation Sample](../../developer_samples/Healthcare/healthcare_openvino.ipynb) for this demo
- [Jupyter* Notebook Samples](https://devcloud.intel.com/edge/advanced/sample_applications/) - additional sample applications 
- [Jupyter* Notebook Tutorials](https://devcloud.intel.com/edge/get_started/tutorials) -  tutorials on using and creating Jupyter* Notebooks
- [Intel® Distribution of OpenVINO™ toolkit Main Page](https://software.intel.com/openvino-toolkit) - learn more about the tools and use of the Intel® Distribution of OpenVINO™ toolkit for implementing inference on the edge


<p style=background-color:#0071C5;color:white;padding:0.5em;display:table-cell;width:100pc;vertical-align:middle>
<img style=float:right src="https://devcloud.intel.com/edge/static/images/svg/IDZ_logo.svg" alt="Intel DevCloud logo" width="150px"/>
<a style=color:white>Intel® DevCloud for the Edge</a><br>   
<a style=color:white href="#top">Top of Page</a> | 
<a style=color:white href="https://devcloud.intel.com/edge/static/docs/terms/Intel-DevCloud-for-the-Edge-Usage-Agreement.pdf">Usage Agreement (Intel)</a> | 
<a style=color:white href="https://devcloud.intel.com/edge/static/docs/terms/Colfax_Cloud_Service_Terms_v1.3.pdf">Service Terms (Colfax)</a>
</p>